K core or K degenerate or coloring number refers to a subset where each vertex has a degree of at least K.

For details, you can check out this video.

https://www.youtube.com/watch?v=rHVrgbc_3JA

In [1]:
import copy

In [2]:
#new functions are created for this script
#including vertex degree, adjacency matrix and remove vertex
class graph:
    def __init__(self):
        self.graph={}
        self.visited={}        

    def append(self,vertexid,edge,weight):
        if vertexid not in self.graph.keys():          
            self.graph[vertexid]={}
            self.visited[vertexid]=0
        self.graph[vertexid][edge]=weight

    def reveal(self):
        return self.graph
    
    def vertex(self):
        return list(self.graph.keys())

    def edge(self,vertexid):
        return list(self.graph[vertexid].keys())
    
    def weight(self,vertexid,edge):
        
        return (self.graph[vertexid][edge])
    
    def size(self):
        return len(self.graph)
    
    def visit(self,vertexid):
        self.visited[vertexid]=1
    
    def go(self,vertexid):
        return self.visited[vertexid]
    
    def route(self):
        return self.visited
    
    #compute the degree of a given vertex
    def degree(self,vertexid):
        return len(self.graph[vertexid])
    
    #convert to adjacency matrix
    def mat(self):
        
        self.matrix=[]
        
        for i in self.graph:

            self.matrix.append([0 for k in range(len(self.graph))])
            
            #when two vertices are connected
            #we set the value to 1
            for j in self.graph[i].keys():        
                self.matrix[i-1][j-1]=1
        
        return self.matrix
    
    #remove one vertex and its edges
    def remove(self,node):
        
        for i in self.graph[node].keys():
            self.graph[i].pop(node)
        self.graph.pop(node)

In [3]:
#to get k core/degenerate
#we need an undirected graph
#in another word, vertices with edge connections 
#are mutually connected to each other
df=graph()
df.append(1,2,0)
df.append(1,3,0)
df.append(1,4,0)
df.append(1,6,0)
df.append(2,1,0)
df.append(3,1,0)
df.append(4,1,0)
df.append(6,1,0)

df.append(2,3,0)
df.append(2,4,0)
df.append(2,6,0)
df.append(3,2,0)
df.append(4,2,0)
df.append(6,2,0)

df.append(3,4,0)
df.append(3,7,0)
df.append(4,3,0)
df.append(7,3,0)

df.append(4,5,0)
df.append(5,4,0)

df.append(5,6,0)
df.append(6,5,0)

In [4]:
df.reveal()

{1: {2: 0, 3: 0, 4: 0, 6: 0},
 2: {1: 0, 3: 0, 4: 0, 6: 0},
 3: {1: 0, 2: 0, 4: 0, 7: 0},
 4: {1: 0, 2: 0, 3: 0, 5: 0},
 6: {1: 0, 2: 0, 5: 0},
 7: {3: 0},
 5: {4: 0, 6: 0}}

![alt text](./preview/kcore1.png)

In [5]:
#sort the vertices by their degree in descending order
def sort_by_degree(df):
    
    dic={}
    for i in df.vertex():
        dic[i]=df.degree(i)

    output=[i[0] for i in sorted(dic.items(), key=lambda x:x[1])]
    
    return output[::-1]

In [6]:
#unlike other problems in graph theory
#i struggle to find a popular algorithm for degeneracy ordering
#hence, i implement my own algorithm to find a k core graph
#basically i am using priority queue to find k core by brute force
#the priority queue is solved by two lists
#one as a queue, the other as a priority list
def find_kcore(core,df):
    
    #make a subset of the original graph
    subset=copy.deepcopy(df)
    
    #fill the queue with all the vertices in the graph
    #we sort the queue by the degree of each vertex
    #so we can remove the node with the minimum degree first
    queue=sort_by_degree(subset)
    
    #pop a random vertex from the queue
    node=queue.pop()
    
    #initialize the priority list
    priority=set([])
    
    #queue contains all the vertices pending for check
    #when the queue is empty, we finish the traversal
    while queue:
        
        #to create k edges for each vertex in the graph
        #you need at least k+1 vertices
        #as the number of vertices drops below k+1
        #for the sake of efficiency
        #there is no point of going further
        if len(subset.vertex())<core+1:
            return {}
        
        #check the degree of a given node
        #if the degree is smaller than our target core number
        #we will remove the node
        if subset.degree(node)<core:
            
            #if the neighbors of a given node exist in the priority list
            #we will check these adjacent vertices first
            #these vertices are top priority
            #because the vertex adjacent to the deleted node suffers from degree loss
            #if we cannot find any top priority
            #we take a union and settle with lower priority
            if priority.intersection(set(subset.edge(node))):
                priority=priority.intersection(set(subset.edge(node)))
            else:
                priority=priority.union(set(subset.edge(node)))
            
            #sort priority list by the degree of each vertex
            #so we can remove the vertex with the minimum degree first
            priority=set([i for i in sort_by_degree(subset) if i in priority])
            
            subset.remove(node)

        #when we have something in the priority list
        #we always examine those vertices first
        #and these vertices should be removed from the queue as well
        #when we have an empty priority list
        #we get a random node from the queue
        if priority:
            node=priority.pop()
            queue.remove(node)
        else:
            node=queue.pop()       
        
    return subset.reveal()

In [7]:
find_kcore(1,df)

{1: {2: 0, 3: 0, 4: 0, 6: 0},
 2: {1: 0, 3: 0, 4: 0, 6: 0},
 3: {1: 0, 2: 0, 4: 0, 7: 0},
 4: {1: 0, 2: 0, 3: 0, 5: 0},
 6: {1: 0, 2: 0, 5: 0},
 7: {3: 0},
 5: {4: 0, 6: 0}}

![alt text](./preview/kcore1.png)

In [8]:
find_kcore(2,df)

{1: {2: 0, 3: 0, 4: 0, 6: 0},
 2: {1: 0, 3: 0, 4: 0, 6: 0},
 3: {1: 0, 2: 0, 4: 0},
 4: {1: 0, 2: 0, 3: 0, 5: 0},
 6: {1: 0, 2: 0, 5: 0},
 5: {4: 0, 6: 0}}

![alt text](./preview/kcore2.png)

In [9]:
find_kcore(3,df)

{1: {2: 0, 3: 0, 4: 0},
 2: {1: 0, 3: 0, 4: 0},
 3: {1: 0, 2: 0, 4: 0},
 4: {1: 0, 2: 0, 3: 0}}

The tricky part of 3-core graph is vertex 6.

In 2-core graph, vertex 6 has a degree of 3.

However, one of its adjacent vertices, vertex 5, only has a degree of 2.

Once we remove the vertex 5, the degree of vertex 6 drops to 2.

Thus, we remove vertex 6 as well.

![alt text](./preview/kcore3.png)

In [10]:
find_kcore(4,df)

{}